In [28]:
import pandas as pd
from ETLPipeline.extract.get_data_from_kaggle import download_crime_dataset, download_mental_health_dataset
from ETLPipeline.transform.transform_functions import transform_country_names
from ETLPipeline.load.mysql import create_database, create_connection

### Extract data from source (Kaggle)

In [29]:
crime_dataset_path = download_crime_dataset() + "/crime-trends-and-operations-of-criminal-justice-systems-un-cts-csv-1.csv"
mental_health_dataset_path = download_mental_health_dataset() + "/Mental health Depression disorder Data.csv"

In [30]:
# Load csv files into pandas dataframes
crime_df = pd.read_csv(crime_dataset_path)
mental_health_df = pd.read_csv(mental_health_dataset_path, index_col=0, low_memory=False)

### Transform

Remove second dataset that is a part of the mental health disorders one


In [31]:
new_dataset_index = mental_health_df[mental_health_df['Entity'] == 'Entity'].index[0]
mental_health_df = mental_health_df.loc[:new_dataset_index - 1]

Remove unneeded columns, and coerce other columns to ensure numeric

In [32]:
mhd_columns_to_remove = ["Code", "Bipolar disorder (%)", "Eating disorders (%)"]
mental_health_df = mental_health_df.drop(columns=mhd_columns_to_remove)

crime_df = crime_df.drop(columns=["sexual violence"])

mhd_numeric_columns = ["Year", "Schizophrenia (%)", "Anxiety disorders (%)", "Drug use disorders (%)", "Depression (%)", "Alcohol use disorders (%)"]
crime_numeric_columns = ["date", "rate"]

# Convert mental health columns to numeric
for col in mhd_numeric_columns:
    mental_health_df[col] = pd.to_numeric(mental_health_df[col], errors="coerce")


# Convert crime to numeric
for col in crime_numeric_columns:
    crime_df[col] = pd.to_numeric(crime_df[col], errors="coerce")

Rename crime's columns to ensure they match mental health dataset

In [33]:
crime_df.rename(columns={"country/territory": "Entity", "date":"Year", "rate": "Sexual Violence Rate"}, inplace=True)

Update the names of the territories to ensure both datasets share the same country names


In [34]:
crime_df = crime_df.apply(transform_country_names, axis=1)

Concatenate the 'England' and 'Wales' rows into a new one

In [35]:
# Get averaged values for each year
filtered = mental_health_df[mental_health_df['Entity'].isin(['England', 'Wales'])]
filtered = filtered.groupby('Year').mean(numeric_only=True).reset_index()
# Drop original England and Wales rows
mental_health_df = mental_health_df[~mental_health_df['Entity'].isin(['England', 'Wales'])]

In [36]:
# Append the filtered DF to the mental health one 
filtered["Entity"] = "United Kingdom (England and Wales)"
mental_health_df = pd.concat([mental_health_df, filtered]).reset_index(drop=True)

Combine our crime and mental health datasets on Entity and year to exclude all the data not present in both

In [37]:
crime_df = crime_df.dropna()
res = pd.merge(mental_health_df, crime_df, on=["Entity", "Year"])

In [ ]:
from collections import defaultdict
c = defaultdict(set)
for i, row in res.iterrows():
    c[row["Entity"]].add(row["Year"])


defaultdict(set,
            {'Albania': {2005, 2006, 2007, 2008, 2009, 2010, 2011},
             'Algeria': {2006, 2007, 2008, 2009, 2010, 2011},
             'Argentina': {2007, 2008},
             'Armenia': {2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011},
             'Australia': {2008, 2009, 2010, 2011},
             'Austria': {2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011},
             'Azerbaijan': {2007, 2008, 2009, 2010},
             'Bahamas': {2005, 2006, 2007, 2008, 2009, 2010, 2011},
             'Bahrain': {2007, 2008},
             'Barbados': {2009},
             'Belarus': {2004, 2005, 2006, 2007, 2008, 2009},
             'Belgium': {2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011},
             'Belize': {2009, 2010, 2011},
             'Bolivia': {2005, 2006, 2007, 2008, 2009, 2010, 2011},
             'Bosnia and Herzegovina': {2007, 2008, 2009, 2010, 2011},
             'Brazil': {2006, 2007, 2008, 2009, 2010, 2011},
             'Bulgaria': {2003,
          

Load

In [39]:
# database = "DS2002FinalData"
# table_name = "MentalHealthAndSACrimeRate"

In [40]:
# create_database(database)

In [41]:
# engine = create_connection(database)

In [42]:
# res.to_sql(table_name, con=engine, if_exists='replace', index=False)